# CIC-IDS2017 Cleaning Pipeline (with column drops)
This notebook discovers CSV files, loads them robustly, applies a cleaning pipeline , and drop explicit columns, then saves the cleaned outputs to a custom folder.

**For Team who Develop Model:**
1. Code for x,y split is availabel in the last part.
2. Download the cleaned dataset file, and apply the path for loading dataset.


In [79]:
from pathlib import Path
import os, glob, re
import pandas as pd
import numpy as np

# --- Configure paths ---
INPUT_DIR = Path('.')  # change to your Kaggle input folder if needed
OUT_DIR   = Path('artifacts_cicids2017_clean_fe')
KEEP_FRACS = {'BENIGN': 1/3}
FLOAT_DECIMALS = 5

# Ensure output directory exists
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('Working directory:', os.getcwd())
print('INPUT_DIR =', INPUT_DIR.resolve())
print('OUT_DIR   =', OUT_DIR.resolve())

Working directory: /Users/jasmine/Documents/CSCI7783_Information Security/Project1_Data/CIC-IDS2017-00
INPUT_DIR = /Users/jasmine/Documents/CSCI7783_Information Security/Project1_Data/CIC-IDS2017-00
OUT_DIR   = /Users/jasmine/Documents/CSCI7783_Information Security/Project1_Data/CIC-IDS2017-00/artifacts_cicids2017_clean_fe


In [ ]:
# Recursively discover CSV files under INPUT_DIR
pattern = str(INPUT_DIR / '**/*.csv')
dspaths = sorted(glob.glob(pattern, recursive=True))

print(f'Found {len(dspaths)} CSV file(s).')
for p in dspaths[:5]:
    print(' -', p)

assert dspaths, 'No CSVs found – check INPUT_DIR and filename pattern.'

Found 6 CSV file(s).
 - archive/Benign-Monday.csv
 - archive/Bruteforce-Tuesday.csv
 - archive/DoS-Wednesday.csv
 - archive/Infiltration-Webattacks-Thursday.csv
 - archive/Portscan-DDos-Botnet-Friday.csv


In [81]:
def reduce_float_precision(df, decimals=4):
    num_cols = df.select_dtypes(include=[np.number]).columns
    if len(num_cols):
        df[num_cols] = df[num_cols].round(decimals)
    return df

def safe_read_csv(path, **kwargs):
    """Robust CSV reader with fallbacks for encoding and separators."""
    try:
        return pd.read_csv(path, **kwargs)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding='latin1', **{k:v for k,v in kwargs.items() if k!='encoding'})
    except pd.errors.ParserError:
        for sep in [',', ';', '\t', '|']:
            try:
                return pd.read_csv(path, sep=sep, **{k:v for k,v in kwargs.items() if k!='sep'})
            except Exception:
                pass
        raise

individual_dfs = []
for p in dspaths:
    try:
        df = safe_read_csv(p)
        individual_dfs.append(df)
    except Exception as e:
        print(f'[skip] {p}: {e}')

print(f'Loaded {len(individual_dfs)} DataFrame(s).')
assert individual_dfs, 'No dataframes loaded – check file accessibility and formats.'

individual_dfs[0].head()

Loaded 6 DataFrame(s).


,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-03 11:56:34.157427,119719148,231,...,17.0,12685486.0,5.296658e+06,20694308.0,6499982.0,-1,-1,0,BENIGN,-1
1,2,192.168.10.9-192.168.10.3-123-123-17,192.168.10.9,123,192.168.10.3,123,17,2017-07-03 11:56:55.428911,65511209,6,...,1506210.0,64004884.0,0.000000e+00,64004884.0,64004884.0,-1,-1,0,BENIGN,-1
2,3,192.168.10.12-224.0.0.251-5353-5353-17,192.168.10.12,5353,224.0.0.251,5353,17,2017-07-03 11:57:21.057686,113976922,267,...,14.0,25498178.0,1.883305e+07,48523116.0,5463561.0,-1,-1,0,BENIGN,-1
3,4,192.168.10.12-152.2.133.52-123-123-17,192.168.10.12,123,152.2.133.52,123,17,2017-07-03 11:57:31.568196,67037196,8,...,11034681.0,55956316.0,0.000000e+00,55956316.0,55956316.0,-1,-1,0,BENIGN,-1
4,5,192.168.10.12-45.79.11.217-123-123-17,192.168.10.12,123,45.79.11.217,123,17,2017-07-03 11:57:30.571719,68045057,8,...,11043596.0,56943904.0,0.000000e+00,56943904.0,56943904.0,-1,-1,0,BENIGN,-1


In [82]:
# Columns to be dropped (original/raw names as they may appear in CIC-IDS2017 CSVs)
drop_columns = [
    "id",
    "Flow ID",
    "Source IP", "Src IP",
    "Source Port", "Src Port",
    "Destination IP", "Dst IP",
    "Timestamp",
    "Attempted Category",
]

def _normalize_name(name: str) -> str:
    """Normalize a column name the same way as our cleaning step."""
    name = name.strip()
    name = re.sub(r'\s+', '_', name)
    name = re.sub(r'[^0-9a-zA-Z_]', '', name)
    return name

# Pre-compute a normalized drop list to match post-renaming columns
drop_columns_normalized = [_normalize_name(c) for c in drop_columns]
drop_columns, drop_columns_normalized[:8]  # sanity peek

(['id',
  'Flow ID',
  'Source IP',
  'Src IP',
  'Source Port',
  'Src Port',
  'Destination IP',
  'Dst IP',
  'Timestamp',
  'Attempted Category'],
 ['id',
  'Flow_ID',
  'Source_IP',
  'Src_IP',
  'Source_Port',
  'Src_Port',
  'Destination_IP',
  'Dst_IP'])

In [83]:

from collections import Counter
import pandas as pd

cnt = Counter()
for df in individual_dfs:
    if 'Label' in df.columns:
        cnt.update(df['Label'].dropna().astype(str))

pd.Series(cnt).sort_values(ascending=False)

BENIGN                                    1587280
Portscan                                   159066
DoS Hulk                                   158449
DDoS                                        95144
Infiltration - Portscan                     67072
DoS GoldenEye                                7567
Botnet - Attempted                           4067
DoS Slowloris                                3998
FTP-Patator                                  3972
DoS Slowhttptest - Attempted                 3367
SSH-Patator                                  2961
DoS Slowhttptest                             1741
DoS Slowloris - Attempted                    1708
Web Attack - Brute Force - Attempted         1292
Botnet                                        736
Web Attack - XSS - Attempted                  655
DoS Hulk - Attempted                          581
DoS GoldenEye - Attempted                      80
Web Attack - Brute Force                       73
Infiltration - Attempted                       45


In [84]:
# --- Cleaning pipeline 
def clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Drop columns using original/raw names (before renaming), if present
    to_drop_raw = [c for c in df.columns if c in set(drop_columns)]
    if to_drop_raw:
        print('Dropping raw columns:', to_drop_raw)
        df = df.drop(columns=to_drop_raw, errors='ignore')

    # 2) Standardize column names
    df.columns = (
        df.columns
        .str.strip()
        .str.replace('\s+', '_', regex=True)
        .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )

    # 3) Drop columns using normalized names (after renaming), if present
    to_drop_norm = [c for c in df.columns if c in set(drop_columns_normalized)]
    if to_drop_norm:
        print('Dropping normalized columns:', to_drop_norm)
        df = df.drop(columns=to_drop_norm, errors='ignore')

    # 4) Replace infinite values with NaN
    df = df.replace([np.inf, -np.inf], np.nan)

    # 5) Downcast numerics
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')

    # 6) Drop duplicate rows
    before = len(df)
    df = df.drop_duplicates()
    after = len(df)
    if before != after:
        print(f'Dropped {before - after} duplicate row(s).')

    # 7) Drop columns with too many NaNs
    na_ratio = df.isna().mean()
    drop_cols = na_ratio[na_ratio > 0.95].index.tolist()
    if drop_cols:
        print('Dropping high-NA columns (>95% NA):', drop_cols)
        df = df.drop(columns=drop_cols, errors='ignore')

    # 8) Fill NaNs (simple strategy)
    num_cols = df.select_dtypes(include=[np.number]).columns
    obj_cols = df.select_dtypes(exclude=[np.number]).columns
    if len(num_cols) > 0:
        df[num_cols] = df[num_cols].fillna(df[num_cols].median())
    if len(obj_cols) > 0:
        df[obj_cols] = df[obj_cols].fillna('')

    return df

# Test on one DF
_test = clean_df(individual_dfs[0])
_test.head()

Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 20691 duplicate row(s).


,Dst_Port,Protocol,Flow_Duration,Total_Fwd_Packet,Total_Bwd_packets,Total_Length_of_Fwd_Packet,Total_Length_of_Bwd_Packet,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,...,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,ICMP_Code,ICMP_Type,Total_TCP_Flow_Time,Label
0,0,0,119719148,231,0,0.0,0.0,0.0,0.0,0.000000,...,22509459.0,17.0,12685486.0,5.296658e+06,20694308.0,6499982.0,-1,-1,0,BENIGN
1,123,17,65511209,6,6,288.0,288.0,48.0,48.0,48.000000,...,1506210.0,1506210.0,64004884.0,0.000000e+00,64004884.0,64004884.0,-1,-1,0,BENIGN
2,5353,17,113976922,267,0,20447.0,0.0,153.0,37.0,76.580521,...,10983883.0,14.0,25498178.0,1.883305e+07,48523116.0,5463561.0,-1,-1,0,BENIGN
3,123,17,67037196,8,8,384.0,384.0,48.0,48.0,48.000000,...,11034681.0,11034681.0,55956316.0,0.000000e+00,55956316.0,55956316.0,-1,-1,0,BENIGN
4,123,17,68045057,8,8,384.0,384.0,48.0,48.0,48.000000,...,11043596.0,11043596.0,56943904.0,0.000000e+00,56943904.0,56943904.0,-1,-1,0,BENIGN


In [132]:
# --- keep exactly these 12 features (+ Label) ---
FEATURES_12 = [
    "Dst_Port",
    "Flow_Duration",
    "Total_Fwd_Packet",
    "Total_Bwd_packets",
    "Total_Length_of_Fwd_Packet",
    "Total_Length_of_Bwd_Packet",
    "Packet_Length_Variance",
    "Bwd_Packet_Length_Std",
    "Packet_Length_Max",
    "Packet_Length_Min",
    "Bwd_Packet_Length_Min",
    "Fwd_Packet_Length_Max",]
KEEP = set(FEATURES_12 + ["Label"])

# -------- 1) read each file keeping only 12 cols (+Label) and MERGE -> cleaned.csv --------
all_dfs = []
for i, df in enumerate(individual_dfs):
    print(f'Processing DataFrame {i+1}/{len(individual_dfs)}...')
    df = clean_df(df)
    missing = KEEP - set(df.columns)
    if missing:
        print(f'  [skip] Missing columns: {missing}')
        continue
    df = df[list(KEEP)]  # keep only desired columns
    all_dfs.append(df)
    print(f'  -> shape: {df.shape}')
print(f'Total valid DataFrames to merge: {len(all_dfs)}')


Processing DataFrame 1/6...
Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 20691 duplicate row(s).
  -> shape: (350933, 13)
Processing DataFrame 2/6...
Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 14705 duplicate row(s).
  -> shape: (307373, 13)
Processing DataFrame 3/6...
Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 18494 duplicate row(s).
  -> shape: (478147, 13)
Processing DataFrame 4/6...
Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 44976 duplicate row(s).
  -> shape: (317100, 13)
Processing DataFrame 5/6...
Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 87639 duplicate row(s).
  -> shape: (459918, 13)
Processing DataFrame 6/6...
  [skip] Mis

Choose the features,merge and then compress the data file

In [ ]:
#merge all, save as cleaned.csv
merged_df = pd.concat(all_dfs, ignore_index=True)
print('Merged DataFrame shape:', merged_df.shape)
merged_df = reduce_float_precision(merged_df, decimals=FLOAT_DECIMALS)
outpath_cleaned = OUT_DIR / 'cicids2017_cleaned.csv'
merged_df.to_csv(outpath_cleaned, index=False)

#compressed version
outpath_cleaned_gz = OUT_DIR / 'cicids2017_cleaned.csv.zip'
merged_df.to_csv(outpath_cleaned_gz, index=False, compression='zip')
print('Saved cleaned data to:', outpath_cleaned)
print('Saved compressed cleaned data to:', outpath_cleaned_gz)




Merged DataFrame shape: (1913471, 13)
Saved cleaned data to: artifacts_cicids2017_clean_fe/cicids2017_cleaned.csv
Saved compressed cleaned data to: artifacts_cicids2017_clean_fe/cicids2017_cleaned.csv.zip


In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split

# Clean one of the individual DataFrames to define "cleaned"
cleaned = clean_df(individual_dfs[0])

# Extract features, selecting specific columns
x = cleaned.drop(["Label"], axis=1)
x = x[['Flow_Duration','Dst_Port','Total_Length_of_Fwd_Packet','Total_Length_of_Bwd_Packet','Total_Fwd_Packet','Total_Bwd_packets','Active_Max','Active_Min','Fwd_Packet_Length_Max']]

y = cleaned['Label'].values
ss = StandardScaler()
x = ss.fit_transform(x)  # Standardize the features

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Dropping raw columns: ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']
Dropped 20691 duplicate row(s).


## Summary
- Discovered CSVs under `INPUT_DIR` (recursive).
- Loaded data robustly with encoding/sep fallbacks.
- Applied a cleaning pipeline with **explicit drops** for identifiers and PII-like columns:
  - `id`, `Flow ID`, `Source IP`/`Src IP`, `Source Port`/`Src Port`, `Destination IP`/`Dst IP`, `Destination Port`/`Dst Port`, `Timestamp`, `Attempted Category`
- Wrote cleaned CSVs into `OUT_DIR` with `_cleaned.csv` suffix.
- (Optional) Produced a merged CSV for convenience.

> Column dropping is resilient to both raw names **and** normalized names.
